#**Níveis de Analise Linguistica e Análise Lexical e Morfológica Automatizada - Tokenização, Stemming e Lematização**

In [12]:
"""
Construa um algoritmo para análise morfológica automatizada utilizando a linguagem Python.
O algoritmo deverá receber um texto (cadeia de caracteres) e devolver uma lista de itens, cada qual contendo: 
a forma textual do token, seu lema, sua classe gramatical e flexões (gênero, número, tempo, etc.).
 - O tagset da tarefa de etiquetagem morfossintática (PoS) será o da Universal Dependencies - o mesmo do exmplo no corpus bosque apresentado
  - As flexões a serem consideradas serão:
     - NOUN: Gênero (Gender) e Número (Number)
     - ADJ: Gênero (Gender) e Número (Number)
     - VERB: Gênero (Gender), Pessoa (Person), Tempo (Tense)  e Forma Verbal (VerbForm)
     - PRON: Gênero (Gender) e Número (Number), Tipo (PronType)
     - DET: Gênero (Gender) e Número (Number)
"""

'\nConstrua um algoritmo para análise morfológica automatizada utilizando a linguagem Python.\nO algoritmo deverá receber um texto (cadeia de caracteres) e devolver uma lista de itens, cada qual contendo: \na forma textual do token, seu lema, sua classe gramatical e flexões (gênero, número, tempo, etc.).\n - O tagset da tarefa de etiquetagem morfossintática (PoS) será o da Universal Dependencies - o mesmo do exmplo no corpus bosque apresentado\n  - As flexões a serem consideradas serão:\n     - NOUN: Gênero (Gender) e Número (Number)\n     - ADJ: Gênero (Gender) e Número (Number)\n     - VERB: Gênero (Gender), Pessoa (Person), Tempo (Tense)  e Forma Verbal (VerbForm)\n     - PRON: Gênero (Gender) e Número (Number), Tipo (PronType)\n     - DET: Gênero (Gender) e Número (Number)\n'

In [13]:
# %pip install conllu
# !wget http://marlovss.work.gd:8080/tomorrow/aula2/bosque.conllu

In [14]:
import conllu
import itertools as it

class AttributeDict(dict):
    __getattr__ = dict.__getitem__
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


class CoNLLU:
   def __init__(self, files):
      self.words = []
      self.sentences = []
      for f in files:
         parsed = conllu.parse(open(f, encoding="utf8").read())
         sents = [[AttributeDict(form = token['form'], lemma=token['lemma'],pos=token['upos'],feats=token['feats']) for token in tokenlist if token['upos']!='_'] for tokenlist in parsed]
         self.sentences.extend(sents)
         self.words.extend([word for sent in sents for word in sent])
      self.pos_tags = set([word.pos for word in self.words])
      self.feats_dict ={pos:set(it.chain.from_iterable([list(word.feats.keys()) for word in self.words if word.pos==pos and word.feats!= None])) for pos in self.pos_tags}

bosque = CoNLLU(files=["bosque.conllu"])

In [23]:
# # Identificação de todos os sufixos e prefixos de tamanho 5

# from nltk.probability import FreqDist

# suffixes = set([word.form.lower()[-5:] for word in bosque.words])
# prefixes = set([word.form.lower()[:5] for word in bosque.words])

In [24]:
# Em função da demora para rodar o código acima, decidi salvar os resultados em arquivos para uso posterior

# with open("suffixes.txt", "w", encoding="utf-8") as f:
#     for suf in suffixes:
#         f.write(suf + "\n")

# with open("prefixes.txt", "w", encoding="utf-8") as f:
#     for pref in prefixes:
#         f.write(pref + "\n")

In [17]:
# Carregando os resultados salvos
with open("suffixes.txt", "r", encoding="utf-8") as f:
    suffixes = set([line.strip() for line in f])

with open("prefixes.txt", "r", encoding="utf-8") as f:
    prefixes = set([line.strip() for line in f])

In [18]:
# Função para análise de sufixos. Para cada sufixo na lista de sufixos, retorna uma lista de dicionários 
# com a classe gramatical e as flexões de cada sufixo
def analyze_suff():
   analyzed = []
   for suf in suffixes:
      for word in bosque.words:
         if word.form.lower()[-5:] == suf:
            analyzed.append({suf: [word.pos, word.feats]})
   return analyzed
            
# Função para análise de prefixos. Para cada prefixo de tamanho qtd_char na lista de prefixos, 
# retorna uma lista de dicionários com o lema de cada prefixo
def analyze_pref(qtd_char):
   analyzed = []
   for pref in prefixes:
      for word in bosque.words:
         if word.form.lower()[:qtd_char] == pref:
            analyzed.append({pref: [word.lemma]})
   return analyzed   

# Sufixos e prefixos (com 5, 4 e 3 caracteres) analisados
# suffixes_analyzed = analyze_suff()
# prefixes_analyzed_5 = analyze_pref(5)
# prefixes_analyzed_4 = analyze_pref(4)
# prefixes_analyzed_3 = analyze_pref(3)

In [19]:
# Em função da demora para rodar o código acima, decidi salvar a análise dos 
# sufixos e prefixos em arquivos para uso posterior
import json

# with open("suffixes_analyzed.json", "w", encoding="utf-8") as f:
#     json.dump(suffixes_analyzed, f)

# with open("prefixes_analyzed_5.json", "w", encoding="utf-8") as f:
#     json.dump(prefixes_analyzed_5, f)

# with open("prefixes_analyzed_4.json", "w", encoding="utf-8") as f:
#     json.dump(prefixes_analyzed_4, f)

# with open("prefixes_analyzed_3.json", "w", encoding="utf-8") as f:
#     json.dump(prefixes_analyzed_3, f)


# Carregando os sufixos e prefixos (com 5, 4 e 3 caracteres) analisados

with open("suffixes_analyzed.json", "r", encoding="utf-8") as f:
    suffixes_analyzed = json.load(f)

with open("prefixes_analyzed_5.json", "r", encoding="utf-8") as f:
    prefixes_analyzed_5 = json.load(f)

with open("prefixes_analyzed_4.json", "r", encoding="utf-8") as f:
    prefixes_analyzed_4 = json.load(f)

with open("prefixes_analyzed_3.json", "r", encoding="utf-8") as f:
    prefixes_analyzed_3 = json.load(f)

In [20]:
# Função para análise de um token. Recebe um token e retorna uma lista com a forma textual do token,
# seu lema, sua classe gramatical e flexões (gênero, número, tempo, etc.)
# Certamente, a função poderia ser otimizada, devo revisá-la posteriormente
def tag_token(token):
    # Inicializa a lista de tags com o token
    tags = [token]

    # Inicializa as variáveis para sufixo e prefixo (com 5, 4 e 3 caracteres)
    token_suf = token.lower()[-5:]
    token_pref_5 = token.lower()[:5]
    token_pref_4 = token.lower()[:4]
    token_pref_3 = token.lower()[:3]

    for pref_item in prefixes_analyzed_5:
        # Se o prefixo de 5 caracteres do token estiver na lista de prefixos analisados, 
        # adiciono o lema do token à lista de tags e paro a busca
        if token_pref_5 in pref_item:
            tags.append(pref_item[token_pref_5][0])
            break
    # Se não encontrar o prefixo de 5 caracteres, tento com o de 4 caracteres
    if len(tags) == 1:
        for pref_item in prefixes_analyzed_4:
            if token_pref_4 in pref_item:
                tags.append(pref_item[token_pref_4][0])
                break
    # Se não encontrar o prefixo de 4 caracteres, tento com o de 3 caracteres
    if len(tags) == 1:
        for pref_item in prefixes_analyzed_3:
            if token_pref_3 in pref_item:
                tags.append(pref_item[token_pref_3][0])
                break
    # Se não encontrar nenhum prefixo, adiciono um espaço em branco à lista de tags
    if len(tags) == 1:
        tags.append('_')

    # Para cada sufixo na lista de sufixos analisados, se o sufixo do token estiver na lista de sufixos analisados,
    # adiciono a classe gramatical e as flexões do token à lista de tags
    for suf_item in suffixes_analyzed:
        if token_suf in suf_item:
            tags.append(suf_item[token_suf][0])
            tags.append(suf_item[token_suf][1])
            break
        
    # Se não encontrar o sufixo, adiciono um espaço em branco à lista de tags
    if len(tags) < 3:
        tags.append('_')
        tags.append('_')
    
    return tags


# Função para análise de uma lista de tokens. Recebe uma lista de tokens e retorna uma lista de itens,
# cada qual contendo a forma textual do token, seu lema, sua classe gramatical e flexões (gênero, número, tempo, etc.)
def tag(tokens):
   tagged = []
   for token in tokens:
      tagged.append((tag_token(token)))
   
   return tagged

In [21]:
test = CoNLLU(files=["test.conllu"])
test_sents = [[word.form for word in sent] for sent in test.sentences]
gold = [[(word.form, word.lemma, word.pos, word.feats) for word in sent] for sent in test.sentences]
predicted = []

# Analisando as sentenças de teste
i = 1
for sent in test_sents:
   print(i, "sentença analisada de", len(test_sents))
   i += 1
   predicted.append((tag(sent)))

1 sentença analisada de 1167
2 sentença analisada de 1167
3 sentença analisada de 1167
4 sentença analisada de 1167
5 sentença analisada de 1167
6 sentença analisada de 1167
7 sentença analisada de 1167
8 sentença analisada de 1167
9 sentença analisada de 1167
10 sentença analisada de 1167
11 sentença analisada de 1167
12 sentença analisada de 1167
13 sentença analisada de 1167
14 sentença analisada de 1167
15 sentença analisada de 1167
16 sentença analisada de 1167
17 sentença analisada de 1167
18 sentença analisada de 1167
19 sentença analisada de 1167
20 sentença analisada de 1167
21 sentença analisada de 1167
22 sentença analisada de 1167
23 sentença analisada de 1167
24 sentença analisada de 1167
25 sentença analisada de 1167
26 sentença analisada de 1167
27 sentença analisada de 1167
28 sentença analisada de 1167
29 sentença analisada de 1167
30 sentença analisada de 1167
31 sentença analisada de 1167
32 sentença analisada de 1167
33 sentença analisada de 1167
34 sentença analisa

In [22]:
def accuracy_lemma(predicted,gold):
   acertos = len([predicted[i][j][1] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][1]==gold[i][j][1]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

def accuracy_pos(predicted,gold):
   acertos = len([predicted[i][j][2] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][2]==gold[i][j][2]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

def accuracy_feats(predicted,gold):
   acertos = len([predicted[i][j][3] for i in range(len(gold)) for j in range(len(gold[i])) if predicted[i][j][3]==gold[i][j][3]])
   totais = sum([len(sent) for sent in gold])
   return acertos/totais

print(accuracy_lemma(predicted,gold))
print(accuracy_pos(predicted,gold))
print(accuracy_feats(predicted,gold))

0.7961527314881901
0.8219823214026952
0.805499203014056
